<h1> Project 1: Finding Lane Lines on the Road </h1>                               <h1> Basil Omar ALfaqeer </h1> 
<h1> Challenge </h1>
<ul>Tools :
<li>Color Selection</li>
<li>Grayscaling</li>
<li>Gaussian Smoothing</li>
<li>Canny edge detection</li>
<li>Region of interest selection</li>
<li>Hough Transform line detection</li>
</ul>

In [23]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Gaussian Blur
def apply_gaussian_blur(gray, kernel_size = 5):
    return gaussian_blur(gray, kernel_size)

Next we define parameters for the Canny function, which will take in an image, a low_threshold, and a high_threshold
<li>pixel values below low_threshold are discarded</li>
<li>pixel values above high_threshold are considered to be edges</li>
<li>pixel values between low and high thresholds are kept if they are touching a pixel above high threshold (edges)</li>

<h3>Canny Input</h3><h5>blur_gray (shape: (540,960)</h5>
<h3>Canny Output</h3><h5>edges (shape:(540, 960) </h5>


In [25]:
# Define Parameters for Canny
def apply_canny(blur_gray, low_threshold = 75, high_threshold = 255):
    # Apply Canny, changes pixel values to be either 0 (black) or 255 (white)
    return canny(blur_gray, low_threshold, high_threshold)

<h3>Image Mask / Region of interest</h3>
<p>Next we wish to apply an image mask to only consider lane lines in a surrounding polygon</p>
<p>Here we will use the <strong>cv2.fillPoly(mask, vertices, ignore_mask_color) function </strong></p>
<li>Input edges shape: (540, 960) Output: masked Image shape: (540, 960) </li>
<li>Function does not change input 'edges'</li>

In [26]:
def apply_mask(edges, image, gray):
    #make general shape of mask
    mask = np.zeros_like(edges)
    
    # create ignore mask color = 255
    ignore_mask_color = 255 

    # create vertices for region
    imshape = image.shape

    ysize = image.shape[0]
    xsize = image.shape[1]
    
    x_middle = xsize/2
    x_offset = 80
    
    y_middle = ysize/2
    y_offset = 90
    vertices = np.array([[(0, ysize),
                         (x_middle - x_offset, y_middle + y_offset),
                         (x_middle + x_offset, y_middle + y_offset),
                         (xsize, ysize)]], dtype=np.int32)
    

    # Polyfill the mask and do a bitwise_and to keep only edges in the region
    (masked_image, mask) = region_of_interest(edges, vertices)
    return masked_image

def plot_mask(edges, image, masked_image):
    # PLOT
    fig = plt.figure()

    a = fig.add_subplot(3,1,1)
    edgeplot = plt.imshow(edges, cmap="gray")
    a.set_title('Edges')

    a = fig.add_subplot(3,1,2)
    a.set_title('Image')
    edgeplot = plt.imshow(image)

    a = fig.add_subplot(3,1,3)
    a.set_title('Masked Image')
    edgeplot = plt.imshow(masked_image, cmap="gray")
    plt.show()

<h3>Hough Lines</h3>
<li>Now that we have masked image Shape: (540, 960). We can use <strong>lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)</strong> to get a row vector of our lines</li>
<li>Copy the image to draw on <strong>line_image</strong> (row vecto with (x1,y1,x2,y2) as elements).</li>
<li>Iterate through the rows of lines, then grab each element (x1, y1, x2, and y2) and use <strong>cv2.line(<strong>line_image</strong>, pt1, pt2, color, thickness)</strong> to draw lines</li>

In [27]:
def apply_hough(masked_image):
    theta = np.pi/180     
    rho = 1.89
    threshold = 60
    
    min_line_len = 60
    max_line_gap = 200

    # Create a line image with hough lines on it
    hough_image, lines = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)

#     print('lines: ', lines.shape)
    # line_zero[:,1,0] = lines[:,1,0]

    return hough_image, lines
    # We gave line_image color when we copied the shape of mask_image and then added a 3rd dimension for our color space.
    # Then in draw_lines we input color = [255, 0, 0] so that will be red in this case. 
    # I changed the input color in draw_lines to [0, 255, 0] so we get green 

def plot_hough(hough_image, lines):
    fig = plt.figure()

    b = fig.add_subplot(2,1,1)
    edgeplot = plt.imshow(lines)
    b.set_title('Lines')

    b = fig.add_subplot(2,2,2)
    b.set_title('Hough Image')
    edgeplot = plt.imshow(hough_image)
    plt.show()

Now we have the line image. And we have the masked image/masked edges
The next step is to draw the line_image ontop of the masked_image
<li>Shape of masked_image: (540, 960)</li>
<li>Shape of line_image: (540,960, 3)...This one has color</li> 
<h5>Plot of line image vs mask image</h5>

In [28]:
def plot_mask_and_hough(masked_image, hough_image):
    fig = plt.figure()
    c = fig.add_subplot(1,2,1)
    edgeplot = plt.imshow(masked_image, cmap="gray")
    c.set_title('masked_image')

    c = fig.add_subplot(1,2,2)
    c.set_title('Hough_image')
    edgeplot = plt.imshow(hough_image)
    plt.show()

In [29]:
# Img is the output of hough lines ==> lines_image
# initial_image is the color_edges, which is the initial edges image with a 3rd dimension for color
# Edges shape: (540, 960)
def apply_lines_edges_weighted(image, edges, hough_image):
    lines_edges_weighted = weighted_img(hough_image, image)
    return lines_edges_weighted

def plot_lines_edges_weighted_hough_and_original():
    # PLOT
    fig = plt.figure()
    d = fig.add_subplot(3,1,1)
    line_edgeplot = plt.imshow(hough_image)
    d.set_title('hough_image')

    d = fig.add_subplot(3,1,2)
    line_edgeplot = plt.imshow(color_edges)
    d.set_title('color_edges')

    d = fig.add_subplot(3,1,3)
    line_edgeplot = plt.imshow(lines_edges_weighted)
    d.set_title('lines_edges_weighted')

    plt.show()
    
def plot_weighted_image(lines_edges_weighted):
    plt.imshow(lines_edges_weighted)

In [30]:
# Converts image to grayscale
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Applies canny transform
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

# Applies gaussian noise kernel, typically use kernel size of 3 or 5. (odd numbers only)
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:  #if it is 3 or 4 channels; RGB or RGBT
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count  #so you can fill multiple colors
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    # Because we just filled the mask matrix with vertices in the *fillPoly* function.
    masked_image = cv2.bitwise_and(img, mask)
    return (masked_image, mask)


def draw_lines(img, lines, color=[255, 0, 0], thickness=12):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x1_values = lines[:, :, 0]
    y1_values = lines[:, :, 1]
    x2_values = lines[:, :, 2]
    y2_values = lines[:, :, 3]


    global prev_left_x_min
    global prev_left_x_max
    global prev_right_x_min
    global prev_right_x_max

    y_max = max(np.amax(y1_values), np.amax(y2_values))
    y_min = max(np.amin(y1_values), np.amin(y2_values))
    left_slopes = []
    right_slopes = []

    right_lane = []
    left_lane = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)

#             if slope < 0:  #left lane has negative slope
            if (-0.9 < slope < -0.6):
                left_slopes.append(slope)
                left_lane.append(line)
            elif 0.45 < slope < 0.7: # right lane positive slope
                right_slopes.append(slope)
                right_lane.append(line)

    left_slopes_arr = np.array(left_slopes)
    right_slopes_arr = np.array(right_slopes)

    left_lines_arr = np.array(left_lane)
    right_lines_arr = np.array(right_lane)

    # Find average slopes
    left_lane_slope_average = np.mean(left_slopes_arr)  # type numpy.float64
    right_lane_slope_average = np.mean(right_slopes_arr)

    y_max = int(y_max)
    y_min = int(y_min)

    if len(left_lines_arr) > 0:
        # Compute Left lane intercept:
        y_intercept_left_lane_average = np.mean(left_lines_arr[:, :, 1] - (left_lines_arr[:, :, 0] * left_lane_slope_average))

        # Compute Left lane Xmax and Xmin:
        x_max_left = int((y_max - y_intercept_left_lane_average) / left_lane_slope_average)
        x_min_left = int((y_min - y_intercept_left_lane_average) / left_lane_slope_average)
            

        cv2.line(img, (x_max_left, y_max), (x_min_left, y_min), color, thickness)
        # Save last coordinates to keep line going
        
        prev_left_x_max = x_max_left
        prev_left_x_min = x_min_left

    else:
        
        if prev_left_x_max is not None:
            print('prev_left_x_max is not None')
            cv2.line(img, (prev_left_x_max, y_max),(prev_left_x_min, y_min), color, thickness)
        else:
            print('nothing')
            
    if len(right_lines_arr) > 0:
        # Compute Right Lane Intercept:
        y_intercept_right_lane_average = np.mean(right_lines_arr[:, :, 1] - (right_lines_arr[:, :, 0] * right_lane_slope_average))

        # Compute Right lane Xmax and Xmin:
        x_max_right = int((y_max - y_intercept_right_lane_average) / right_lane_slope_average)
        x_min_right = int((y_min - y_intercept_right_lane_average) / right_lane_slope_average)

        cv2.line(img, (x_max_right, y_max), (x_min_right, y_min), color, thickness)
        prev_right_x_max = x_max_right
        prev_right_x_min = x_min_right
    else:
        if prev_right_x_max is not None:
            print('prev_right_x_max is not NONE')
#         print("right_lines_arr is empty. Not drawing anything as there are no lines to choose from.")
            cv2.line(img, (prev_right_x_max, y_max), (prev_right_x_min, y_min), color, thickness)
        else:
            
            print('nothing right side empty')




def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        img shape will be 2D output from canny transform
    Returns an image with hough lines drawn.
    """
    hough_lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                                  maxLineGap=max_line_gap)

    # Create a line_img which we will ultimately draw on in our draw lines function
    # line_img will become zeros (540, 960, 3). 
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)  #*before img.shape unpacks the tuple

    draw_lines(line_img, hough_lines)
    return (line_img, hough_lines)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


<h1>Video Processing:</h1>

In [31]:
# Save previous coordinates as global variables in case I don't detect any lines on the current frame
prev_left_x_min = None
prev_left_x_max = None
prev_right_x_min = None
prev_right_x_max = None

In [32]:
def process_image(image):
    gray = grayscale(image)
    blur_gray = apply_gaussian_blur(gray)
    edges = apply_canny(blur_gray)
    masked_image = apply_mask(edges, image, gray)
    hough_image, lines = apply_hough(masked_image)
    lines_edges_weighted = apply_lines_edges_weighted(image, edges, hough_image)
    return lines_edges_weighted

In [33]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

<h1>Challenge</h1>

In [34]:
white_output = 'challenge.mp4'
clip1 = VideoFileClip("challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


MoviePy: building video file challenge.mp4
----------------------------------------

Writing video into challenge.mp4
|----------| 0/241   0% [elapsed: 00:00 left: ?, ? iters/sec]

C:\Users\basil\Miniconda3\envs\carnd-term1\lib\site-packages\moviepy\audio\io\readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))


prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
|----------| 6/241   2% [elapsed: 00:00 left: 00:19, 11.90 iters/sec]

C:\Users\basil\Miniconda3\envs\carnd-term1\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\basil\Miniconda3\envs\carnd-term1\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
|----------| 12/241   4% [elapsed: 00:01 left: 00:19, 11.66 iters/sec]Warning: in file challenge.mp4, 2764800 bytes wanted but 0 bytes read,at frame 12/251, at time 0.48/10.04 sec. Using the last valid frame instead.
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
|----------| 17/241   7% [elapsed: 00:01 left: 00:21, 10.36 iters/sec]Warning: in file challenge.mp4, 2764800 bytes wanted but 0 bytes read,at frame 17/251, at time 0.68/10.04 sec. Using the last valid frame instead.
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
prev_right_x_max is not NONE
|----------| 22/241   9% [elapsed: 00:02 left: 00:22,  9.88 iters/sec]Warning: in file challenge.mp4, 

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))